In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

In [1]:

def load_data():
    df = pd.read_csv('transistors.csv')

    X = df['X'].values
    Y = df['Y'].values
    return X, Y

def plot_data(X, Y):
    plt.scatter(X, Y, color='r')
    plt.xlabel('year')
    plt.ylabel('growth')
    plt.title('Moore\'s Law')
    plt.show()

In [2]:

class regression(object):
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model = nn.Linear(1,1).to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(
                            self.model.parameters(),
                            lr = 0.001
                            )
        
        X, Y = load_data()
        plot_data(X, Y)

        X = X.reshape(-1,1)
        Y = Y.reshape(-1,1)

        ################### Convert Data Distribution to Linear ###################
        X = np.log(X)
        Y = np.log(Y)

        self.X = torch.from_numpy(X.astype(np.float32)).to(self.device)
        self.Y = torch.from_numpy(Y.astype(np.float32)).to(self.device)

    def train(self):
        n_epoches = 30
        losses = []
        for _ in range(n_epoches):
            self.optimizer.zero_grad() # Avoid accumilating gradients

            P = self.model(self.X)
            loss = self.criterion(self.Y, P)

            loss.backward()
            self.optimizer.step()

            loss_np = loss.detach().numpy().squeeze()
            losses.append(loss_np)

        plt.plot(losses)
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.title('Loss')
        plt.show()

    def predict(self):

        P = self.model(self.X)

        # This will leads to an Error
        P1 = P.numpy().squeeze()

        # Method 1
        P1 = P.detach().numpy().squeeze()

        # Method 2
        with torch.no_grad():
            P2 = self.model(self.X).numpy().squeeze()

In [ ]:
regression = regression()
regression.train()